## 1. Converting .stl file to .ply
For that task, the next page will be used [stl to ply](https://imagetostl.com/convert/file/stl/to/ply).

In [ ]:
nombre           = ""                      # .ply file name, which must be in the same place as this notebook
                                           # Example: object is in //ruta/objeto.ply, name variable must be "objeto"

target_num_pts_1 =                         # The objective quantity of points is established (the final value will be approximate), for the case 
                                           # in which points are not necessarily evenly spaced

target_num_pts_2 =                         # The objective quantity of points is established (the final value will be approximate), for the case
                                           # in which points are evenly spaced

error_exactitud  =                         # This way, the exact quantity of points will be between (1-error_exactitud)*target_num_pts and
                                           # (1+error_exactitud)*target_num_pts

## 2. Pertinent definitions.
The next libraries are needed to be installed: pcu, numpy, json, matplotlib.pyplot

### Importing necessary libraries

In [ ]:
import point_cloud_utils as pcu
import numpy as np
import json
import matplotlib.pyplot as plt

### Defining necessary functions that will be used

In [ ]:
def plot_views(point_cloud, sizes=1):
    """
    Plot the 3 different views of the point cloud in (XY, XZ, YZ) perspectives, and a general 3D visualization.

    Parameters:
        point_cloud (dict): The point cloud, as a dictionary.
        sizes (float, int): Visual size of the points in the views.
    """
    points = np.array(point_cloud["vertices"])

    # XY view
    plt.figure()
    plt.scatter(points[:, 0], points[:, 1], s=sizes)
    plt.title("XY View")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.axis("equal")

    # XZ view
    plt.figure()
    plt.scatter(points[:, 0], points[:, 2], s=sizes)
    plt.title("XZ View")
    plt.xlabel("X")
    plt.ylabel("Z")
    plt.axis("equal")

    # YZ view
    plt.figure()
    plt.scatter(points[:, 1], points[:, 2], s=sizes)
    plt.title("YZ View")
    plt.xlabel("Y")
    plt.ylabel("Z")
    plt.axis("equal")

    # 3D perspective
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=sizes)
    ax.view_init(elev=30, azim=45)  
    plt.title("Perspective View")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

    plt.show()

## 3. Extracting the elements from .ply object

In [ ]:
# v is an (n,3) array that contains the n object vertices coordinates
# f is an (m, k) array that contains m faces, each of them conformed by k vertices, (m, 3) if triangular faces, for example
# n is an (m, 3) array that contains m normals of the m faces
v, f, n = pcu.load_mesh_vfn(f"./{nombre}.ply")
target_radius_2 = np.linalg.norm(v.max(0) - v.min(0)) * 0.02 

# Next line has to be uncommented only if a variable type error is outputted later when pcu.interpolate is called 
# f = np.array(f, dtype=np.int32)

## 4. First option, generating a point cloud of a quantity given approximately by target_num_pts
The new point cloud is generated (only points), saved in .ply and in .json, and displayed.

In [ ]:
fid_1, bc_1 = pcu.sample_mesh_poisson_disk(v, f, int(target_num_pts_1*0.9))   # Resampling information is generated, following a Poisson disk distribution
v_resampling_1 = pcu.interpolate_barycentric_coords(f, fid_1, bc_1, v)        # The new point cloud is generated from this previous information     
target_aux_1 = target_num_pts_1

while (v_resampling_1.shape[0] < target_aux_1 * (1-error_exactitud) or v_resampling_1.shape[0] > target_aux_1 * (1+error_exactitud)):
    if v_resampling_1.shape[0] < target_aux_1 * (1-error_exactitud):
        target_num_pts_1 = target_num_pts_1 * (1+error_exactitud)
    elif v_resampling_1.shape[0] > target_aux_1 * (1+error_exactitud):
        target_num_pts_1 = target_num_pts_1 * (1-error_exactitud)
    fid_1, bc_1 = pcu.sample_mesh_poisson_disk(v, f, int(target_num_pts_1*0.9))
    v_resampling_1 = pcu.interpolate_barycentric_coords(f, fid_1, bc_1, v)

print("La nueva nube generada tiene", v_resampling_1.shape[0], "puntos.")        # Exact number of points in the new point cloud is shown 

### Saving the new point cloud (only points, faces information is lost)

In [ ]:
mesh_data_1 = {
    'vertices': v_resampling_1.tolist()
}

# Saving the new dictionary in a .json file (only points, faces information is lost)
with open(f"./{nombre}_{v_resampling_1.shape[0]}_1.json", 'w') as json_file:
    json.dump(mesh_data_1, json_file)

# Saving the new point cloud in a .ply file (only points, faces information is lost)
pcu.save_mesh_v(f"./{nombre}_{v_resampling_1.shape[0]}_1.ply", v_resampling_1)

### Point cloud visualization

In [ ]:
# Uncomment the next two lines if the visualization is from a external .json file and not from the point cloud generated before 
# with open(f"./externo.json", 'r') as f:
#         mesh_data_1 = json.load(f)

plot_views(mesh_data_1)

## 5. Second option. Generating a point cloud, in which points are evenly spaced 
The new point cloud is generated (only points), saved in .ply and in .json, and displayed.

In [ ]:
fid_2,bc_2=pcu.sample_mesh_poisson_disk(v,f,num_samples=-1,radius=target_radius_2) # Resampling information is generated, following a Poisson disk distribution
v_resampling_2 = pcu.interpolate_barycentric_coords(f, fid_2, bc_2, v)             # The new point cloud is generated from this previous information

while (v_resampling_2.shape[0] < target_num_pts_2*(1-error_exactitud) or v_resampling_2.shape[0] > target_num_pts_2*(1+error_exactitud)):
    if v_resampling_2.shape[0] < target_num_pts_2*(1-error_exactitud):
        target_radius_2 = target_radius_2 * (1-error_exactitud)
    elif v_resampling_2.shape[0] > target_num_pts_2*(1+error_exactitud):
        target_radius_2 = target_radius_2 * (1+error_exactitud)
    fid_2, bc_2 = pcu.sample_mesh_poisson_disk(v, f, num_samples=-1, radius=target_radius_2)
    v_resampling_2 = pcu.interpolate_barycentric_coords(f, fid_2, bc_2, v)

print("La nueva nube generada tiene", v_resampling_2.shape[0], "puntos.")          # Exact number of points in the new point cloud is shown

### Saving the new point cloud (only points, faces information is lost)

In [ ]:
mesh_data_2 = {
    'vertices': v_resampling_2.tolist()
}

# Saving the new dictionary in a .json file (only points, faces information is lost)
with open(f"./{nombre}_{v_resampling_2.shape[0]}_2.json", 'w') as json_file:
    json.dump(mesh_data_2, json_file)

# Saving the new point cloud in a .ply file (only points, faces information is lost)
pcu.save_mesh_v(f"./{nombre}_{v_resampling_2.shape[0]}_2.ply", v_resampling_2)

### Point cloud visualization

In [ ]:
# Uncomment the next two lines if the visualization is from a external .json file and not from the point cloud generated before
# with open(f"./externo.json", 'r') as f:
#         mesh_data_2 = json.load(f)

plot_views(mesh_data_2)